# Matrices, their Null Spaces and Regular Markov Chains
#### Bharath Variar, 2019B5A70930H

## Importing Libraries

In [1]:
import numpy as np
from sympy import *

## Powers of a Matrix

### Defining functions

In [2]:
def matrix_multiply(matA, matB):
    """
    Returns the product of 2 matrics
    i/p: The two matrices to be multipled matA(n1 x n2), matB(n3 x n4)
    o/p: If n2 == n4: returns n1 x n4 product matrix
         else: returns -1
    """
    if matA.shape[1] != matB.shape[0]:
        print("Matrix multiplication invalid")
        return -1
    else:
        result_mat = np.zeros((matA.shape[0], matB.shape[1]))
        for i in range(matA.shape[0]):  # matA.shape[0] = number of rows
            row = matA[i]
            for j in range(matB.shape[1]):  # matB.shape[1] = number of columns
                col = matB[:, j]
                dot = 0
                for k in range(len(row)):
                    dot += row[k] * col[k]
                result_mat[i][j] = dot
        return result_mat

In [3]:
def matrix_power(matA, power):
    """
    Returns the input matrix raised to the power 'power'
    i/p: Matrix matA(n1 x n2), and integer power
    o/p: If n1 == n2 and power is an integer: Returns matA^power
         else: returns -1
    """
    if type(power) is not int:
        print("Power is not a valid integer")
        return -1
    result_mat = matA
    for i in range(power - 1):
        result_mat = matrix_multiply(result_mat, matA)
        if type(result_mat) is int:
            return -1
    return result_mat

### Implementation

In [4]:
A = np.random.randint(10, size=(3, 3))
print(f"matA:\n {A}")
B = np.random.randint(10, size=(3, 5))
print(f"matB:\n {B}")

matA:
 [[6 5 4]
 [2 4 6]
 [4 6 1]]
matB:
 [[7 4 5 9 7]
 [9 8 3 0 5]
 [7 0 5 3 8]]


In [5]:
print(f'matA x matB: \n{matrix_multiply(A, B)}')

matA x matB: 
[[115.  64.  65.  66.  99.]
 [ 92.  40.  52.  36.  82.]
 [ 89.  64.  43.  39.  66.]]


In [6]:
print(f"(matA)^3: \n{matrix_power(A, 3)}")

(matA)^3: 
[[752. 954. 750.]
 [540. 696. 586.]
 [552. 718. 513.]]


## Regular Markov Chain as a Matrix

In [7]:
def create_stochastic_matrix(size):
    mat = np.zeros((size, size))
    for i in range(size):
        row_sum = 0
        for j in range(size - 1):
            mat[i][j] = np.random.uniform(0, (1 - row_sum))
            row_sum += mat[i][j]
        mat[i][-1] = 1 - row_sum
    return mat

In [8]:
def check_regular_matrix(matA, iterations= 1000):
    for i in range (1, iterations + 1):
        mat = matrix_power(matA, i)
        if (type(mat) == int): return -1 # matrix_power() returns -1 if error
        mat_size = len(mat) # Matrix has to be a square
        count = 0
        for j in range(mat_size):
            for k in range(mat_size):
                if (mat[j][k] == 0): 
                    break
                else: 
                    count += 1
        if (count == (mat_size ** 2)): 
            print(f"The matrix is regular, and when it is raised to power {i}, it is positive.")
            break
        else:
            print(f"The matrix is not regular upto its {i}th power", end = '\r')
    return

In [9]:
C = np.array([[0.7, 0, 0.3], [0, 1, 0], [0.2, 0, 0.8]]).reshape(3, -1)
print(f"MatC: \n{C}")
check_regular_matrix(C, 1000)

MatC: 
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.2 0.  0.8]]


In [10]:
# Debugging for validity of stochastic matrix
# Generate two random stochastic matrices
for i in range(2):
    b = create_stochastic_matrix(4)
    for i in range(4):
        for j in range(4):
            if (b[i][j] < 0 or b[i][j] > 1):
                print("Not Valid Stochastic Matrix")    
    print(b)
    print()

[[0.83704918 0.01640369 0.01560049 0.13094664]
 [0.52126022 0.11864689 0.03016439 0.3299285 ]
 [0.63595154 0.15285163 0.06243431 0.14876252]
 [0.46937595 0.37644008 0.12789398 0.02629   ]]

[[0.45115462 0.40632942 0.12828932 0.01422664]
 [0.94907791 0.03145267 0.01484003 0.00462939]
 [0.11336622 0.14480769 0.72540452 0.01642156]
 [0.52348111 0.05133453 0.33160788 0.09357649]]



## Null Space of a Matrix
The null space of a matrix A is the same as the null space for its row reduced echelon form (rref(A)).

In [11]:
def rref(matA):
    rref = matA.copy()
    pivot = 0
    rows = len(rref)
    cols = len(rref[0])
    rank = 0
    for r in range(rows):
        if (cols < pivot):
            break
        i = r
        while (rref[i, pivot] == 0):
            i += 1
            if (rows == 1):
                i = r
                pivot += 1
                if (cols == pivot):
                    break
        vec = rref[i]
        rref[i] = rref[r]
        rref[r] = vec
        if (rref[r][pivot] != 0):
            rref[r] = rref[r] / rref[r][pivot]
        for i in range(rows):
            if (i != r):
                rref[i] -= (rref[r]*rref[i][pivot]) 
        pivot += 1
    for row in range(rows):
        for col in range(cols):
            if (rref[row][col] != 0):
                rank += 1
                break
                
    return rref, rank

In [12]:
E = np.array([1, 2, -1, -4, 2, 3, -1, -11, -2, 0, -3, 22]).reshape(3, -1)
print(E)
rref_e, r = rref(E)
print(f"rref(E): \n{rref_e}")
E = Matrix([[1, 2, -1, -4], [2, 3, -1, -11], [-2, 0, -3, 22]])
print (f"sympy.rref(): \n{E.rref()}")

[[  1   2  -1  -4]
 [  2   3  -1 -11]
 [ -2   0  -3  22]]
rref(E): 
[[ 1  0  0 -8]
 [ 0  1  0  1]
 [ 0  0  1 -2]]
sympy.rref(): 
(Matrix([
[1, 0, 0, -8],
[0, 1, 0,  1],
[0, 0, 1, -2]]), (0, 1, 2))


In [13]:
def gauss_elimination(mat):  
    length = len(mat)  
    vec = [0 for i in range(length)]  
    for j in range(length - 1, -1, -1):  
        vec[j] = mat[j][length] / mat[j][j]  
        for k in range(j - 1, -1, -1):  
            mat[k][length] -= mat[k][j] * vec[j]  
    return vec

In [14]:
def null_space(matA):
    '''
    Finds null space of matrix in rref
    matA.X = 0
    '''
    dimension = len(matA[0]) # number of columns
    matA, rankA = rref(matA)
    # Rank-nullity theorem
    if (rankA == dimension):
        np.zeros(dimension)
    # rankA < dimensions
    aug_mat = np.concatenate((matA, np.zeros((len(matA), 1))), axis = 1)
    # Reducing the augmented matrix to rref, and obtaining their ranks
    aug_mat, rank_aug = rref(aug_mat) 
    # rank(matA) == rank(aug_mat) since homogenous system
    for i in range(dimensions - rankA):
        
    return np.linalg.solve(matA, np.zeros(len(matA[0])))

IndentationError: expected an indented block (1989762431.py, line 18)

In [ ]:
F = np.array([[1, 0, 1, 3], [2, 3, 4, 7], [-1, -3, -3, -4]]).reshape(3, -1)
print(f"matF:\n{F}")
null_space_F, r = rref(F)
F = Matrix(F)
print(f"null_space(F): \n {null_space_F}")
print(f"F.nullspace(): \n {F.rref()}")

In [ ]:
G = Matrix([[1, 2, -1, -4], [2, 3, -1, -11], [-2, 0, -3, 22]])
print (G.rref())